# RAG关于抗马拉色菌肽序列的批量判断

In [1]:
import pandas as pd
import random
from pathlib import Path
import os
import sys
sys.path.append(os.path.abspath('/home/qcdong/codes/llamaIndex'))
from utils_llama_index.llm_loader import load_hf_llm
from utils_llama_index.retriever_util import create_vector_store_index_hierarchical_node
from llama_index.core import Settings

data_file = Path('.').absolute().parent.parent / 'data' / 'invention_processed.csv'
df_seq=pd.read_csv(data_file)

#随机抽取10个patent进行演示
random10_patents=random.sample(list(df_seq['patent_no'].unique()),30) #list(df_seq['patent_no'].unique())

/home/qcdong/anaconda3/envs/llaIdx/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["LLAMA_INDEX_CACHE_DIR"] = "/mnt/nas1/models/llama_index_cache"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
Settings.embed_model = "local:/mnt/nas1/models/BAAI/bge-small-en-v1.5"

## Llama2相关组件初始化

In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from IPython.display import Markdown, display

In [2]:
import torch
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate

LLAMA2_7B_chat = "/home/cymei/llama_hf/llama-2-chat-7b-hf"
LLAMA2_13B_chat = "/mnt/sdc/Llama-2-13b-chat-hf"
LLAMA2_7B_hf = "/home/cymei/llama_hf/llama-2-7b-hf"

selected_model = LLAMA2_7B_chat 

SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
)

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=2048,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=selected_model,
    model_name=selected_model,
    device_map="auto",
    # change these settings below depending on your GPU
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True},
)

INFO:numexpr.utils:Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:107: UserWarning: 

BNB_CUDA_VERSION=XXX can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64
Loading CUDA version: BNB_CUDA_VERSION=118


  warn((f'\n\n{"="*80}\n'


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
    Settings
)

from llama_index.core.node_parser import (
    HierarchicalNodeParser,
    get_leaf_nodes,
)

from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers.auto_merging_retriever import AutoMergingRetriever
from llama_index.core.storage import StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore


llm = load_hf_llm()
Settings.llm = llm


class LLM_summary:
    def __init__(self,doc_dic):
        self.doc_dic=doc_dic
    
    def hierarchy_node_index(self,docs,chunk_list=[1024, 512, 256],similarity_top_k=6):
        '''
        获取分层节点，并建立对应索引
        '''
        node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_list)
        nodes = node_parser.get_nodes_from_documents(docs)

        docstore = SimpleDocumentStore()
        # insert nodes into docstore
        docstore.add_documents(nodes)
        # define storage context (will include vector store by default too)
        storage_context = StorageContext.from_defaults(docstore=docstore)

        leaf_nodes = get_leaf_nodes(nodes)
        base_index = VectorStoreIndex(
            leaf_nodes,
            storage_context=storage_context,
        )
        _retriever = base_index.as_retriever(similarity_top_k=similarity_top_k)
        retriever = AutoMergingRetriever(_retriever, storage_context)
        query_engine = RetrieverQueryEngine(retriever)
        return query_engine

2024-04-13 20:59:39.753 | INFO     | utils_llama_index.llm_loader:load_hf_llm:63 - use_8bit =1 max_length = 8192 context_window = 6144 max_new_tokens = 2048


INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 3/3 [00:08<00:00,  2.82s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 批量seq判断

In [5]:
from elasticsearch import Elasticsearch
from llama_index.core.schema import Document
import json

es = Elasticsearch(hosts="http://192.168.1.124:29200")
INDEX = "patents"

# #对于sequence数量过多的patent，将其按一定间隔分段输入prompt
# def cut_prompts(df,total_length=1200,max_num=30,max_seq_len=300):
#     #设置最大prompt长度和最大seq数量，防止prompt占用过多token
#     prompts=[];index_list=[] #用于存储在同一个prompt中的seq index
#     nums=df.shape[0]
#     seq_str='Content describing the functions and applications of following sequence numbers and their peptide sequences:\n'
#     len_now=len(seq_str)
#     num_now=0;indexs=[]
#     for index,row in df.iterrows():
#         new_str='{}:{}\n'.format(row['new_seq_id'],row['peptide sequence'])
#         if(len_now+len(new_str)<=total_length and num_now+1<=max_num):
#             #长度和数量都满足时加入prompt
#             pass
#         else:
#             #有一项不满足就新增prompt
#             prompts.append(seq_str);index_list.append(indexs)
#             seq_str='Content describing the functions and applications of following sequence numbers and their peptide sequences:\n'
#             len_now=len(seq_str)
#             num_now=0;indexs=[]

#         if(len(new_str)<=max_seq_len):
#             #剔除超长的seq_no
#             len_now+=len(new_str)
#             num_now+=1
#             seq_str+=new_str
#             indexs.append(index)

#     if(num_now>0):
#         prompts.append(seq_str);index_list.append(indexs)
#     return prompts,index_list

#使用es查询并写入专利文本
def get_text_es(patent_no):  #,file_path='./temp_data/temp_des.txt'

    patent_num=patent_no[2:-2]
    patent_country=patent_no[:2]
    patent_kind=patent_no[-2:]
    #es查询条件，设置关键值publication_doc_number
    body = {
        "query":{
            "bool":{
                "must":[
                    {"term":{"publication_doc_number": patent_num}},
                    {"term":{"publication_country": patent_country}}, 
                    {"term":{"publication_kind":patent_kind}}               
                ]
            }
        }
    }
    result = es.search(index=INDEX,body=body, request_timeout=60)
    res_dic=dict(result)

    #分开存储claims和description中的内容
    doc_dic={}
    if('claim' in res_dic['hits']['hits'][0]['_source']):
        claims=['claim '+''.join(dic['claim_text']) for dic in res_dic['hits']['hits'][0]['_source']["claim"]]
        text_claims='The claim information:\n'+'\n'.join(claims)
        doc_claims=Document(text=text_claims)
        doc_dic['claims']=doc_claims
    if("Description" in res_dic['hits']['hits'][0]['_source']["description"]):
        text_des='\nThe description information:\n'+'\n'.join(res_dic['hits']['hits'][0]['_source']["description"]["Description"])
        doc_des=Document(text=text_des)
        doc_dic['description']=doc_des
    
    return doc_dic

In [7]:
# cut_prompt version2(加入随机排列的seq序列)
import random

def shuffle_seq(seq_list):
#用于随机排列seq
    random.shuffle(seq_list)
    return ''.join(seq_list)

#对于sequence数量过多的patent，将其按一定间隔分段输入prompt
def cut_prompts(df,total_length=1000,max_num=30,max_seq_len=300,rand_num=5):
    #设置最大prompt长度和最大seq数量以及单个seq的最大长度，防止prompt占用过多token，同时设置每一个prompt中seq随机排列的次数
    prompts=[];index_list=[] #用于存储在同一个prompt中的seq index
    seq_set=[];indexs=[] #存储不同seq的排列组合,存储seq对应的Index
    #当前prompt的长度和seq数量
    len_now=0;num_now=0
    for index,row in df.iterrows():
        new_str='{}:{}\n'.format(row['new_seq_id'],row['peptide sequence'])
        if(len_now+len(new_str)<=total_length and num_now+1<=max_num):
            #长度和数量都满足时加入prompt
            pass
        else:
            #有一项不满足就新增prompt,并将一组seq随机排列多次
            prompts.append([shuffle_seq(seq_set) for i in range(rand_num)])
            index_list.append(indexs)
            seq_set=[];indexs=[]
            len_now=0;num_now=0

        if(len(new_str)<=max_seq_len):
            #剔除超长的seq_no
            len_now+=len(new_str)
            num_now+=1
            seq_set.append(new_str)
            indexs.append(index)
    if(num_now>0):  
        prompts.append([shuffle_seq(seq_set) for i in range(rand_num)])
        index_list.append(indexs)
    return prompts,index_list

In [31]:
df_unit=df_seq[df_seq['patent_no']=='US09169290B2'].copy()
seq_prompts,_=cut_prompts(df_unit,max_num=15)
doc_dic=get_text_es('US09169290B2')
# summarier=LLM_summary(doc_dic)

# engine2=summarier.hierarchy_node_index([summarier.doc_dic['claims'],summarier.doc_dic['description']],similarity_top_k=5)
head_str='Given content describing the functions and applications of following sequence numbers and their peptide sequences:\n'
tail_str="""If some sequence numbers have the ability to inhibit Malassezia fungus, please list them one by one then simply explain the reason in next line. 
    Make sure the target sequence numbers are specific about inhibition of Malassezia and don't make any guesses. 
    Please focuse on the given context and don't use prior knowledge. If none of sequence numbers satisfied, please return 'None' first and explain the reason."""
print(f'{len(seq_prompts) = }')
for seq_set in seq_prompts:
    print(f'{len(seq_set) = }')
    count = 0
    for prompt in seq_set:
        print(prompt)
        count += 1
        if count > 1:
            break
    break

print(len(doc_dic['claims'].text))
print(doc_dic['claims'].text[-200:])
print()
idx = doc_dic['claims'].text.lower().find('malassezia')
if idx > -1:
    print(doc_dic['claims'].text[idx-100:idx+100])
print()
counts = doc_dic['claims'].text.lower().count('malassezia')
print(f'{counts = }')
print(doc_dic['claims'].text)

INFO:elastic_transport.transport:POST http://192.168.1.124:29200/patents/_search [status:200 duration:0.009s]
POST http://192.168.1.124:29200/patents/_search [status:200 duration:0.009s]
len(seq_prompts) = 4
len(seq_set) = 5
SEQ_ID_NO_29:KLTKPKPQAESKKKKK
SEQ_ID_NO_26:RRRRWWRRRR
SEQ_ID_NO_25:RRWRRWRR
SEQ_ID_NO_21:RRWWRR
SEQ_ID_NO_22:RRRWRRR
SEQ_ID_NO_19:RRWRR
SEQ_ID_NO_24:RRRWWRRR
SEQ_ID_NO_17:RWRWR
SEQ_ID_NO_28:GKKEKPEKKVKK
SEQ_ID_NO_31:KKKDKVKK
SEQ_ID_NO_32:KVRQGTLKKAR
SEQ_ID_NO_20:RRWRWR
SEQ_ID_NO_27:RRWWRRWWRR
SEQ_ID_NO_18:RRWWR
SEQ_ID_NO_23:RRWRWRR

SEQ_ID_NO_23:RRWRWRR
SEQ_ID_NO_25:RRWRRWRR
SEQ_ID_NO_19:RRWRR
SEQ_ID_NO_22:RRRWRRR
SEQ_ID_NO_28:GKKEKPEKKVKK
SEQ_ID_NO_20:RRWRWR
SEQ_ID_NO_21:RRWWRR
SEQ_ID_NO_26:RRRRWWRRRR
SEQ_ID_NO_27:RRWWRRWWRR
SEQ_ID_NO_24:RRRWWRRR
SEQ_ID_NO_31:KKKDKVKK
SEQ_ID_NO_29:KLTKPKPQAESKKKKK
SEQ_ID_NO_18:RRWWR
SEQ_ID_NO_17:RWRWR
SEQ_ID_NO_32:KVRQGTLKKAR

3920
laim 1, wherein the peptide consists of D-amino acids.
claim 19. The method of claim 14, wherein the

/tmp/ipykernel_127504/863875574.py:57: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  result = es.search(index=INDEX,body=body, request_timeout=60)
/tmp/ipykernel_127504/863875574.py:57: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = es.search(index=INDEX,body=body, request_timeout=60)


## 单个patent演示

In [26]:
df_unit=df_seq[df_seq['patent_no']=='US09169290B2'].copy()
seq_prompts,_=cut_prompts(df_unit)
doc_dic=get_text_es('US09169290B2')
summarier=LLM_summary(doc_dic)
engine=summarier.hierarchy_node_index([summarier.doc_dic['claims']])
question1="Is this patent describing agents for inhibiting Malassezia fungus? Please make sure the patent is specific about inhibition of Malassezia. Plesae answer 'Yes' or 'No' first, then explain the reason in next line."
response1 = engine.query(question1)
display(Markdown(f"<b>{response1}</b>"))

INFO:elastic_transport.transport:POST http://192.168.1.124:29200/patents/_search [status:200 duration:0.011s]
POST http://192.168.1.124:29200/patents/_search [status:200 duration:0.011s]


/tmp/ipykernel_127504/863875574.py:57: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  result = es.search(index=INDEX,body=body, request_timeout=60)
/tmp/ipykernel_127504/863875574.py:57: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = es.search(index=INDEX,body=body, request_timeout=60)
Batches: 100%|██████████| 1/1 [00:00<00:00, 40.27it/s]

INFO:llama_index.core.retrievers.auto_merging_retriever:> Filling in node. Node id: 7d3cebb4-3a99-43d9-b686-3b0e21794537> Node text: claim 4. The method of claim 3 wherein the non-dermatophyte infection is caused by a fungus selec...

> Filling in node. Node id: 7d3cebb4-3a99-43d9-b686-3b0e21794537> Node text: claim 4. The method of claim 3 wherein the non-dermatophyte infection is caused by a fungus selec...

INFO:llama_index.core.retrievers.auto_merging_retriever:> Merging 4 nodes into parent node.
> Parent node id: 5777eca0-f616-473a-9017-28c56a70c336.
> Parent node text: The claim information:
claim 1. A method for the treatment or alleviation of a fungal infection i...

> Merging 4 nodes into parent node.
> Parent node id: 5777eca0-f616-473a-9017-28c56a70c336.
> Parent node text: The claim information:
claim 1. A method for the treatment or alleviation of a fungal infection i...

INFO:llama_index.core.retrievers.auto_merging_retriever:> Merging 3 nodes into parent node.
> Parent n

<b>Yes, the patent describes agents for inhibiting Malassezia fungus. 

The patent specifically mentions Malassezia species in claim 4, where it lists a variety of fungi, including Malassezia species, for which the method of administering the peptide or pharmaceutically acceptable salt thereof can be used for treatment. Additionally, claim 15 further narrows down the fungi to include Malassezia species among others like Aspergillus species, Fusarium species, Candida species, Alternaria species, Penicillium species, Scopulariopsis species, and Cryptococcus species. This indicates that the patent is indeed focused on inhibiting Malassezia fungus, along with other fungal infections.</b>

In [27]:
for node in response1.source_nodes:
    print(node.text)
    print()

The claim information:
claim 1. A method for the treatment or alleviation of a fungal infection in a subject, comprising administering to the subject a peptide or pharmaceutically acceptable salt thereof, wherein the peptide is selected from the group consisting of:(a) a peptide consisting of a sequence of 11 to 15 contiguous arginine residues; and(b) a peptide consisting of a sequence having 1 or 2 amino acid substitutions to the sequence recited in (a).
claim 2. The method of claim 1 wherein the fungal infection is a dermatophyte infection.
claim 3. The method of claim 1 wherein the fungal infection is a non-dermatophyte infection.
claim 4. The method of claim 3 wherein the non-dermatophyte infection is caused by a fungus selected from the group consisting of Absidia species, Aspergillus species, Cryptococcus species, Malassezia species, Candida species, Rhizomucor species, Saccharomyces species, Hansenula species, Fusarium species, Mucor species, Trichosporon species, Rhodotorula sp

In [29]:
summarier=LLM_summary(doc_dic)

engine2=summarier.hierarchy_node_index([summarier.doc_dic['claims'],summarier.doc_dic['description']],similarity_top_k=5)
head_str='Given content describing the functions and applications of following sequence numbers and their peptide sequences:\n'
tail_str="""If some sequence numbers have the ability to inhibit Malassezia fungus, please list them one by one then simply explain the reason in next line. 
    Make sure the target sequence numbers are specific about inhibition of Malassezia and don't make any guesses. 
    Please focuse on the given context and don't use prior knowledge. If none of sequence numbers satisfied, please return 'None' first and explain the reason."""
print(f'{len(seq_prompts) = }')
for seq_set in seq_prompts:
    print(f'{len(seq_set) = }')
    for prompt in seq_set:
        print(prompt)
        question2=head_str+prompt+tail_str 
        response2=engine2.query(question2)
        print(response2.response)
        print()

Batches: 100%|██████████| 1/1 [00:00<00:00, 44.83it/s]


len(seq_prompts) = 4


Batches: 100%|██████████| 1/1 [00:00<00:00, 63.28it/s]

INFO:llama_index.core.retrievers.auto_merging_retriever:> Merging 2 nodes into parent node.
> Parent node id: 15abb804-efcc-4ef4-96df-00b090515991.
> Parent node text: dRdFdWdWdFdRdRdR-CONH2(SEQ ID NO: 40)NP354ILRWPWWPWRRK-CONH2(SEQ ID NO: 41)NP355dAdKdRdHdHdGdYdKd...

> Merging 2 nodes into parent node.
> Parent node id: 15abb804-efcc-4ef4-96df-00b090515991.
> Parent node text: dRdFdWdWdFdRdRdR-CONH2(SEQ ID NO: 40)NP354ILRWPWWPWRRK-CONH2(SEQ ID NO: 41)NP355dAdKdRdHdHdGdYdKd...

INFO:llama_index.core.retrievers.auto_merging_retriever:> Merging 2 nodes into parent node.
> Parent node id: 582853b6-c18f-49b0-a708-81eb3b70833c.
> Parent node text: 5-2 kDa)NP121Poly-L-Arginine (MW 5-15 kDa)NP213cyclo-KKKKKKK(SEQ ID NO: 1)NP301RVRVR(SEQ ID NO: 2...

> Merging 2 nodes into parent node.
> Parent node id: 582853b6-c18f-49b0-a708-81eb3b70833c.
> Parent node text: 5-2 kDa)NP121Poly-L-Arginine (MW 5-15 kDa)NP213cyclo-KKKKKKK(SEQ ID NO: 1)NP301RVRVR(SEQ ID NO: 2...



Based on the given context information, the following sequence numbers have the ability to inhibit Malassezia fungus:

1. SEQ_ID_NO_22: RRRWRRR
2. SEQ_ID_NO_20: RRWRWR
3. SEQ_ID_NO_25: RRWRRWRR
4. SEQ_ID_NO_26: RRRRWWRRRR
5. SEQ_ID_NO_27: RRWWRRWWRR
6. SEQ_ID_NO_29: KLTKPKPQAESKKKKK

The reason these sequence numbers are effective against Malassezia fungus is that they are part of the peptide sequences tested against various fungal strains, including yeast, in the provided data (Table 2). The Minimum Inhibitory Concentration (MIC) values in the table indicate the effectiveness of these peptides in inhibiting the growth of Malassezia fungus. Lower MIC values represent higher potency in inhibiting the fungal growth. The specific mechanism of action for these peptides against Malassezia fungus is not provided in the context, but it is generally believed that antimicrobial peptides can disrupt the cell membrane of fungal cells, leading to cell death.



...

INFO:llama_index.core.retrievers.auto_merging_retriever:> Merging 2 nodes into parent node.
> Parent node id: 15abb804-efcc-4ef4-96df-00b090515991.
> Parent node text: dRdFdWdWdFdRdRdR-CONH2(SEQ ID NO: 40)NP354ILRWPWWPWRRK-CONH2(SEQ ID NO: 41)NP355dAdKdRdHdHdGdYdKd...

> Merging 2 nodes into parent node.
> Parent node id: 15abb804-efcc-4ef4-96df-00b090515991.
> Parent node text: dRdFdWdWdFdRdRdR-CONH2(SEQ ID NO: 40)NP354ILRWPWWPWRRK-CONH2(SEQ ID NO: 41)NP355dAdKdRdHdHdGdYdKd...

INFO:llama_index.core.retrievers.auto_merging_retriever:> Merging 2 nodes into parent node.
> Parent node id: 582853b6-c18f-49b0-a708-81eb3b70833c.
> Parent node text: 5-2 kDa)NP121Poly-L-Arginine (MW 5-15 kDa)NP213cyclo-KKKKKKK(SEQ ID NO: 1)NP301RVRVR(SEQ ID NO: 2...

> Merging 2 nodes into parent node.
> Parent node id: 582853b6-c18f-49b0-a708-81eb3b70833c.
> Parent node text: 5-2 kDa)NP121Poly-L-Arginine (MW 5-15 kDa)NP213cyclo-KKKKKKK(SEQ ID NO: 1)NP301RVRVR(SEQ ID NO: 2...



Based on the given context information, the following sequence numbers have the ability to inhibit Malassezia fungus:

1. SEQ_ID_NO_20: RRWRWR
2. SEQ_ID_NO_19: RRWRR
3. SEQ_ID_NO_22: RRRWRRR
4. SEQ_ID_NO_25: RRWRRWRR
5. SEQ_ID_NO_23: RRWRWRR
6. SEQ_ID_NO_26: RRRRWWRRRR
7. SEQ_ID_NO_29: KLTKPKPQAESKKKKK

The reason these sequence numbers have the ability to inhibit Malassezia fungus is that they are listed in Table 2, which shows the activity of the peptides against various fungal strains, including yeast. The table provides Minimum Inhibitory Concentration (MIC) data for different peptides against Candida albicans strains, which are a type of Malassezia fungus. The lower the MIC value, the more potent the peptide is in inhibiting the growth of the fungus. The sequence numbers listed above have MIC values ranging from 1024 μM to 2048 μM, indicating their potential to inhibit the growth of Malassezia fungi.



Batches: 100%|██████████| 1/1 [00:00<00:00, 64.77it/s]

INFO:llama_index.core.retrievers.auto_merging_retriever:> Merging 2 nodes into parent node.
> Parent node id: 15abb804-efcc-4ef4-96df-00b090515991.
> Parent node text: dRdFdWdWdFdRdRdR-CONH2(SEQ ID NO: 40)NP354ILRWPWWPWRRK-CONH2(SEQ ID NO: 41)NP355dAdKdRdHdHdGdYdKd...

> Merging 2 nodes into parent node.
> Parent node id: 15abb804-efcc-4ef4-96df-00b090515991.
> Parent node text: dRdFdWdWdFdRdRdR-CONH2(SEQ ID NO: 40)NP354ILRWPWWPWRRK-CONH2(SEQ ID NO: 41)NP355dAdKdRdHdHdGdYdKd...

INFO:llama_index.core.retrievers.auto_merging_retriever:> Merging 2 nodes into parent node.
> Parent node id: 582853b6-c18f-49b0-a708-81eb3b70833c.
> Parent node text: 5-2 kDa)NP121Poly-L-Arginine (MW 5-15 kDa)NP213cyclo-KKKKKKK(SEQ ID NO: 1)NP301RVRVR(SEQ ID NO: 2...

> Merging 2 nodes into parent node.
> Parent node id: 582853b6-c18f-49b0-a708-81eb3b70833c.
> Parent node text: 5-2 kDa)NP121Poly-L-Arginine (MW 5-15 kDa)NP213cyclo-KKKKKKK(SEQ ID NO: 1)NP301RVRVR(SEQ ID NO: 2...



KeyboardInterrupt: 

In [72]:
from llama_index.response.notebook_utils import display_source_node
for node in response2.source_nodes:
    display_source_node(node, source_length=20000)
    print("--------")

**Node ID:** a86ba698-9002-42aa-b5f9-c801fea71929<br>**Similarity:** 0.8938498559513711<br>**Text:** The best activity observed for SEQ ID NO:1 was against E. faecium (MIC50/90 of 1/2 μg/mL), Acinetobacter (MIC50/90 of 2/4 μg/mL), and E. coli (MIC50/90 of 2/4 μg/mL).
Example 3: Disruption of Biofilms--P. aeruginosa or S. aureus cells were grown in vinyl microtiter plates in Mueller-Hinton broth for 24 h at 37° C. to allow a mature biofilm to form. After about 24 hrs, an exemplary peptide of SEQ ID NO:1 diluted 2-fold in MHB was added to the biofilm containing wells and incubated for 1 hour. The well-associated biomass was stained with crystal violet and quantified by measuring absorbance at 550 nm following solubilization in 30% acetic acid.FIG. 13 shows the effect on biofilm produced by P. aeruginosa by contacting the biofilm with the exemplary peptide of SEQ ID NO:1, as determined by the absorbance at 550 nm.<br>

--------


**Node ID:** ad685347-11c0-4d61-a759-aa4d41d7587e<br>**Similarity:** 0.8923386063950121<br>**Text:** 12. For SEQ ID NO:1, the majority of the results fall within a narrow range, in contrast to those of the comparators.TABLE 10DrugMIC rangeModeMIC50MIC90% R1EscherichiaSEQ ID NO: 1     1-8224—coliCeftolozane-   0.06->320.120.2514.9(n = 101)TazobactamPiperacillin-   0.25->12822648.9TazobactamMeropenem≤0.008-40.0150.0150.031Tobramycin   0.25->640.50.53221.7Ceftazidime   0.06->320.250.5>3233.6Colistin ≤0.03-0.50.120.120.120Levofloxacin  0.015->8>84>846.5Table 11 summarizes the activity of SEQ ID NO:1 against the various resistance phenotypes in this study.<br>

--------


**Node ID:** ee7bccca-785d-4632-8896-229189d7e882<br>**Similarity:** 0.891451385524864<br>**Text:** Exemplary peptides screened include RRWVRRVRRVWRRVVRVVRRWVRR (SEQ ID NO:1); IRRRRRRIRRRRRR (SEQ ID NO:2); IRRRIRRIRRRIRRIRRRIRR (SEQ ID NO:3); IRRIIRRIRRIIRRIRRIIRR (SEQ ID NO:4); VWRWVRRVWRWVRRVWRWVRR (SEQ ID NO: 5); VWRWVRRVWRWVRR (SEQ ID NO:6); VVRVVRRVVRVVRR (SEQ ID NO:7); and VVRVVRVVVRVVRVVVRVVRV (SEQ ID NO:8).Each peptide was formulated in PBS buffer prior to testing. Peptides of SEQ ID NO:5 and SEQ ID NO:8 were brought up in 100% DMSO with 0.002% polysorbate-80 at 1.28 mg/mL (40× the highest final test concentration of 32 mcg/mL).<br>

--------


**Node ID:** d3967235-c2a1-4862-9e73-b0cb8ddfddb8<br>**Similarity:** 0.8880393716272598<br>**Text:** A 0.1 mL volume of each dilution was plated on TSA and incubated at 37° C. overnight for enumeration of developed colonies. Toxicity was evaluated on the basis of the presence of peptide-related adverse effects such as signs of inflammation, weight loss and presence of bacteria in the blood and tissues. Throughout the course of the infection or at the disease endpoint, animals were euthanized and tissues weighed and homogenized using 70 mm cell strainers to determine bacterial cfu/g tissue.FIG. 19A and FIG. 19B show that SEQ ID NO:1 effectively eradicated the infection at both the 2 mg/kg and 4 mg/kg dose. SEQ ID NO:1 demonstrated protection from E. coli cUTI infection at 2 mg/kg and 4 mg/kg as a 50 mg/kg dose of Gentamicin.
Example 8: Determination of Bacterial Resistance--Antimicrobial resistance as a function of time of multiple drug resistant organisms for the exemplary peptide of SEQ ID NO:1, Rifampicin, LL-37, and Colistin was determined. Each antibiotic was compared for their propensity to select for bacterial resistance phenotypes in vitro.<br>

--------


**Node ID:** 34a2020b-65b5-41e2-878b-20948da5f7a4<br>**Similarity:** 0.8854702585832758<br>**Text:** The MIC was read and recorded as the lowest concentration of peptide that inhibited visible growth of the organism.The results of the bacterial inhibition study are depicted in Table 2.TABLE 2MIC (μg/mL)Formula Formula Formula Formula Formula Formula Formula SEQABDE1E3E4GIDSEQSEQSEQSEQSEQSEQSEQNO.IDIDIDIDIDIDIDIsolate1NO: 2NO: 3NO: 4NO: 5NO: 6NO: 7NO: 8ColistinLevofloxacinMeropenemStaphylococcus81688160.5>32>32>320.2510.12aureus(0.06-0.5)2 (0.03-0.12)ATCC29213(MSSA; QC)Staphylococcus4168480.5>3232>320.25>4aureusATCC43300(MRSA)Staphylococcus488480.5>32>32>328>4aureusNRS382(USA100;HA-MRSA)Streptococcus16>323216328>32>32>321 0.<br>

--------


## 批量判断 version1

In [20]:
import re
temp_path='./temp_data/'

df_res=None
for patent in random10_patents:
    df_unit=df_seq[df_seq['patent_no']==patent].copy()
    seq_prompts,index_list=cut_prompts(df_unit) #,max_num=10
    #查询并存储文本
    doc_dic=get_text_es(patent)

    #使用llama-index建立查询引擎
    summarier=LLM_summary(doc_dic)
    #第一个问题只用claims中的内容
    engine=summarier.hierarchy_node_index([summarier.doc_dic['claims']],similarity_top_k=5)

    #首先让模型判断专利整体是否与antifungal有关
    question1="Is this patent describing agents (e.g. molecules) for inhibiting Malassezia fungus? Such applications should be claimed in the “Claims” of the patent. Please make sure the patent is specific about inhibition of Malassezia.Plesae answer 'Yes' or 'No' first, then explain the reason in next line."
    response1 = engine.query(question1)
    if('yes' in response1.response.lower()):
        df_unit['is_patent_antifungal']=1
    else:
        df_unit['is_patent_antifungal']=0
    df_unit['Q1_reason']=response1.response
    df_unit['Q2_reason']=''

    #其次依次判断每一部分seq是否antifungal
    antifungal_seqs=[]
    #第一问为yes时才进行第二问
    if('yes' in response1.response.lower()):
        #第二个问题用claims+description的内容
        engine2=summarier.hierarchy_node_index([summarier.doc_dic['claims'],summarier.doc_dic['description']],similarity_top_k=5)  #如果仅用claims，会有更多无关seq被推测成正确的
        for seq_index,prompt in enumerate(seq_prompts):
            question2=prompt+"Please determine which sequence numbers have the ability to inhibit Malassezia fungus? Make sure the target sequence numbers are specific about inhibition of Malassezia and don't make any guesses. Please list them one by one then simply explain the reason in next line. If any sequence number isn't satisfied, just explain the reason."
            response2=engine2.query(question2)

            #使用正则表达式找出回答中的seq
            target_format=re.findall('[0-9]+[. ]+SEQ_ID_NO_[0-9]+[: ]+[A-Z]+\n',response2.response)
            # target_seqs=re.findall('SEQ_ID_NO_[0-9]+',''.join(target_format))
            if(target_format==[]):
                #在回答出现特殊情况时再使用简单正则表达式
                target_seqs=re.findall('SEQ_ID_NO_[0-9]+',response2.response)  #简单的正则匹配可能存在问题，需要进行修改。
            else:
                #防止模型在输出符合条件seq的同时，多嘴输出不符合条件的seq
                order=[]
                for index,seq in enumerate(target_format):
                    index_unit=seq.split('.')[0]
                    if(index_unit in order):
                        #如果有重复的序号出现，说明回答中额外列举了不符条件的seq，需要截断剔除
                        target_format=target_format[:index]
                        break
                    order.append(index_unit)
                target_seqs=re.findall('SEQ_ID_NO_[0-9]+',''.join(target_format))

            antifungal_seqs+=target_seqs  
            df_unit['Q2_reason'].loc[index_list[seq_index]]=response2.response
    #标出符合条件的seq
    df_unit['is_seq_antifungal']=df_unit.apply(lambda row: 1 if(row['new_seq_id'] in antifungal_seqs) else 0,axis=1)

    if(df_res is None):
        df_res=df_unit
    else:
        df_res=pd.concat([df_res,df_unit],axis=0)
    print('-------------------------------------next one!--------------------------------------------------------')

INFO:elastic_transport.transport:POST http://192.168.1.124:29200/patents/_search [status:200 duration:0.005s]
POST http://192.168.1.124:29200/patents/_search [status:200 duration:0.005s]


INFO:llama_index.retrievers.auto_merging_retriever:> Merging 3 nodes into parent node.
> Parent node id: f084bc1b-a936-4019-bc13-7582c858f51a.
> Parent node text: The claim information:
claim 1. A method for the treatment or alleviation of a fungal infection i...

> Merging 3 nodes into parent node.
> Parent node id: f084bc1b-a936-4019-bc13-7582c858f51a.
> Parent node text: The claim information:
claim 1. A method for the treatment or alleviation of a fungal infection i...

> Merging 3 nodes into parent node.
> Parent node id: f084bc1b-a936-4019-bc13-7582c858f51a.
> Parent node text: The claim information:
claim 1. A method for the treatment or alleviation of a fungal infection i...

INFO:llama_index.retrievers.auto_merging_retriever:> Merging 2 nodes into parent node.
> Parent node id: 80a73149-c542-4dfb-9094-254247f81b31.
> Parent node text: claim 9. The method of claim 7 wherein the systemic mycosis is an opportunistic mycosis selected ...

> Merging 2 nodes into parent node.
> Pare

/tmp/ipykernel_536630/4100429343.py:59: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  result = es.search(index=INDEX,body=body, request_timeout=60)
/tmp/ipykernel_536630/4100429343.py:59: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = es.search(index=INDEX,body=body, request_timeout=60)
/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/genera

> Merging 2 nodes into parent node.
> Parent node id: 7f1e2d48-06d9-4063-8a8e-cc8f52dc4e9e.
> Parent node text: The claim information:
claim 1. A method for the treatment or alleviation of a fungal infection i...

-------------------------------------next one!--------------------------------------------------------
INFO:elastic_transport.transport:POST http://192.168.1.124:29200/patents/_search [status:200 duration:0.024s]
POST http://192.168.1.124:29200/patents/_search [status:200 duration:0.024s]


/tmp/ipykernel_536630/4100429343.py:59: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  result = es.search(index=INDEX,body=body, request_timeout=60)
/tmp/ipykernel_536630/4100429343.py:59: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = es.search(index=INDEX,body=body, request_timeout=60)


...

> Merging 2 nodes into parent node.
> Parent node id: 6f65e70f-40d2-4deb-b05f-aa506376e9ff.
> Parent node text: The claim information:
claim 1. A method of treating alopecia, the method comprising:externally a...

-------------------------------------next one!--------------------------------------------------------
INFO:elastic_transport.transport:POST http://192.168.1.124:29200/patents/_search [status:200 duration:0.029s]
POST http://192.168.1.124:29200/patents/_search [status:200 duration:0.029s]


/tmp/ipykernel_536630/4100429343.py:59: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  result = es.search(index=INDEX,body=body, request_timeout=60)
/tmp/ipykernel_536630/4100429343.py:59: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = es.search(index=INDEX,body=body, request_timeout=60)


INFO:llama_index.retrievers.auto_merging_retriever:> Merging 2 nodes into parent node.
> Parent node id: 0e820cc8-2495-4ea1-b14a-805dda26ca1d.
> Parent node text: The claim information:
claim 1. A method of treating alopecia, the method comprising: externally ...

> Merging 2 nodes into parent node.
> Parent node id: 0e820cc8-2495-4ea1-b14a-805dda26ca1d.
> Parent node text: The claim information:
claim 1. A method of treating alopecia, the method comprising: externally ...

> Merging 2 nodes into parent node.
> Parent node id: 0e820cc8-2495-4ea1-b14a-805dda26ca1d.
> Parent node text: The claim information:
claim 1. A method of treating alopecia, the method comprising: externally ...

INFO:llama_index.retrievers.auto_merging_retriever:> Merging 3 nodes into parent node.
> Parent node id: 481e9837-15c2-4c66-90ce-b6715a928c17.
> Parent node text: claim 9. The method of treating alopecia according to claim 1, wherein the content of the C-type ...

> Merging 3 nodes into parent node.
> Pare

/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


-------------------------------------next one!--------------------------------------------------------


## 批量判断 version2

In [12]:
import re
temp_path='./temp_data/'
rand_times=5 #设置随机排序次数
df_res=None
for patent in random10_patents:
    df_unit=df_seq[df_seq['patent_no']==patent].copy()
    seq_prompts,index_list=cut_prompts(df_unit,max_num=15,rand_num=rand_times) #,max_num=10
    #查询并存储文本
    doc_dic=get_text_es(patent)

    #使用llama-index建立查询引擎
    summarier=LLM_summary(doc_dic)
    #第一个问题只用claims中的内容
    engine=summarier.hierarchy_node_index([summarier.doc_dic['claims']],similarity_top_k=5)

    #首先让模型判断专利整体是否与antifungal有关
    question1="Is this patent describing agents (e.g. molecules) for inhibiting Malassezia fungus? Such applications should be claimed in the “Claims” of the patent. Please make sure the patent is specific about inhibition of Malassezia.Plesae answer 'Yes' or 'No' first, then explain the reason in next line."
    response1 = engine.query(question1)
    if('yes' in response1.response.lower()):
        df_unit['is_patent_antifungal']=1
    else:
        df_unit['is_patent_antifungal']=0
    df_unit['Q1_reason']=response1.response

    #其次依次判断每一部分seq是否antifungal
    #第一问为yes时才进行第二问
    if('yes' in response1.response.lower()):
        #第二个问题用claims+description的内容
        engine2=summarier.hierarchy_node_index([summarier.doc_dic['claims'],summarier.doc_dic['description']],similarity_top_k=5)  #如果仅用claims，会有更多无关seq被推测成正确的
        head_str='Given content describing the functions and applications of following sequence numbers and their peptide sequences:\n'
        tail_str="""If some sequence numbers have the ability to inhibit Malassezia fungus, please list them one by one then simply explain the reason in next line. 
            Make sure the target sequence numbers are specific about inhibition of Malassezia and don't make any guesses. 
            Please focuse on the given context and don't use prior knowledge. If none of sequence numbers satisfied, please return 'None' first and explain the reason."""

        freq_dic={} #存储每个seq的出现频率
        for seq_set in seq_prompts: 
            for prompt in seq_set:
                question2=head_str+prompt+tail_str
                response2=engine2.query(question2)

                # #如果模型返回结果中有none，说明没有符合条件的seq，不进行后续步骤
                # if('none' in response2.response.lower()):
                #     continue

                #使用正则表达式找出回答中的seq
                target_format=re.findall('[0-9]+[. ]+SEQ_ID_NO_[0-9]+[: ]+[A-Z]+\n',response2.response)
                if(target_format==[]):
                    #在回答出现特殊情况时再使用简单正则表达式
                    target_seqs=re.findall('SEQ_ID_NO_[0-9]+',response2.response)  #简单的正则匹配可能存在问题，需要进行修改。
                else:
                    #防止模型在输出符合条件seq的同时，多嘴输出不符合条件的seq
                    order=[]
                    for index,seq in enumerate(target_format):
                        index_unit=seq.split('.')[0]
                        if(index_unit in order):
                            #如果有重复的序号出现，说明回答中额外列举了不符条件的seq，需要截断剔除
                            target_format=target_format[:index]
                            break
                        order.append(index_unit)
                    target_seqs=re.findall('SEQ_ID_NO_[0-9]+',''.join(target_format))
                #去重
                target_seqs=list(set(target_seqs))

                #更新各个seq的出现频率
                for seq in target_seqs:
                    if(seq not in freq_dic):
                        freq_dic[seq]=1
                    else:
                        freq_dic[seq]+=1

    #标出符合条件的seq,符合条件的概率
    df_unit['is_seq_antifungal_prob']=df_unit.apply(lambda row: freq_dic[row['new_seq_id']]/rand_times if(row['new_seq_id'] in freq_dic) else 0,axis=1)

    if(df_res is None):
        df_res=df_unit
    else:
        df_res=pd.concat([df_res,df_unit],axis=0)
    print('-------------------------------------next one!--------------------------------------------------------')

INFO:elastic_transport.transport:POST http://192.168.1.124:29200/patents/_search [status:200 duration:0.008s]
POST http://192.168.1.124:29200/patents/_search [status:200 duration:0.008s]


/tmp/ipykernel_52853/2421638550.py:59: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  result = es.search(index=INDEX,body=body, request_timeout=60)
/tmp/ipykernel_52853/2421638550.py:59: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = es.search(index=INDEX,body=body, request_timeout=60)


INFO:llama_index.retrievers.auto_merging_retriever:> Merging 3 nodes into parent node.
> Parent node id: e81f6fc0-9342-4a99-bed7-f8bcbb70274a.
> Parent node text: The method of claim 20, wherein the endotoxins are bacterial endotoxins or fungal endotoxins.
cla...

> Merging 3 nodes into parent node.
> Parent node id: e81f6fc0-9342-4a99-bed7-f8bcbb70274a.
> Parent node text: The method of claim 20, wherein the endotoxins are bacterial endotoxins or fungal endotoxins.
cla...

> Merging 3 nodes into parent node.
> Parent node id: e81f6fc0-9342-4a99-bed7-f8bcbb70274a.
> Parent node text: The method of claim 20, wherein the endotoxins are bacterial endotoxins or fungal endotoxins.
cla...



/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


INFO:llama_index.retrievers.auto_merging_retriever:> Merging 2 nodes into parent node.
> Parent node id: cec08bc1-3cdb-4a62-945c-65792940877c.
> Parent node text: Accordingly, n may be between about 1.5 to 5. In one example, n may be 1.5, 2, 2.5, 3, 3.5, 4, 4....

> Merging 2 nodes into parent node.
> Parent node id: cec08bc1-3cdb-4a62-945c-65792940877c.
> Parent node text: Accordingly, n may be between about 1.5 to 5. In one example, n may be 1.5, 2, 2.5, 3, 3.5, 4, 4....

> Merging 2 nodes into parent node.
> Parent node id: cec08bc1-3cdb-4a62-945c-65792940877c.
> Parent node text: Accordingly, n may be between about 1.5 to 5. In one example, n may be 1.5, 2, 2.5, 3, 3.5, 4, 4....

INFO:llama_index.retrievers.auto_merging_retriever:> Merging 2 nodes into parent node.
> Parent node id: be3c7e87-1a84-4e53-8c28-4decea1242dc.
> Parent node text: In one example, the peptide as described herein when represented by general formula X1Y1X2Y2X3Y3X...

> Merging 2 nodes into parent node.
> Pare

...

-------------------------------------next one!--------------------------------------------------------
INFO:elastic_transport.transport:POST http://192.168.1.124:29200/patents/_search [status:200 duration:0.048s]
POST http://192.168.1.124:29200/patents/_search [status:200 duration:0.048s]


/tmp/ipykernel_52853/2421638550.py:59: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  result = es.search(index=INDEX,body=body, request_timeout=60)
/tmp/ipykernel_52853/2421638550.py:59: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = es.search(index=INDEX,body=body, request_timeout=60)


INFO:llama_index.retrievers.auto_merging_retriever:> Merging 2 nodes into parent node.
> Parent node id: 9d7dc331-3c63-4d0e-bf73-1b42a0df2743.
> Parent node text: claim 70. The pharmaceutical formulation of claim 67, wherein the surfactant is selected from the...

> Merging 2 nodes into parent node.
> Parent node id: 9d7dc331-3c63-4d0e-bf73-1b42a0df2743.
> Parent node text: claim 70. The pharmaceutical formulation of claim 67, wherein the surfactant is selected from the...

> Merging 2 nodes into parent node.
> Parent node id: 9d7dc331-3c63-4d0e-bf73-1b42a0df2743.
> Parent node text: claim 70. The pharmaceutical formulation of claim 67, wherein the surfactant is selected from the...

INFO:llama_index.retrievers.auto_merging_retriever:> Merging 2 nodes into parent node.
> Parent node id: 9aad4906-a5ac-46fa-b027-9db967e5bce4.
> Parent node text: claim 77. The pharmaceutical formulation of claim 74, wherein the surfactant is selected from the...

> Merging 2 nodes into parent node.
> Pare

/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/cymei/miniconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/cymei/minicon

INFO:llama_index.retrievers.auto_merging_retriever:> Merging 2 nodes into parent node.
> Parent node id: 4871c78b-da2d-4009-82a8-e5d39a28d2e9.
> Parent node text: The best activity observed for SEQ ID NO:1 was against E. faecium (MIC50/90 of 1/2 μg/mL), Acinet...

> Merging 2 nodes into parent node.
> Parent node id: 4871c78b-da2d-4009-82a8-e5d39a28d2e9.
> Parent node text: The best activity observed for SEQ ID NO:1 was against E. faecium (MIC50/90 of 1/2 μg/mL), Acinet...

> Merging 2 nodes into parent node.
> Parent node id: 4871c78b-da2d-4009-82a8-e5d39a28d2e9.
> Parent node text: The best activity observed for SEQ ID NO:1 was against E. faecium (MIC50/90 of 1/2 μg/mL), Acinet...

-------------------------------------next one!--------------------------------------------------------


In [13]:
df_res[df_res['is_patent_antifungal']==1].to_csv('Malassezia_patents_res_13b_new.csv',index=False)

In [19]:
#输入prompt中的节点
#做embedding相似度匹配的只有query中的内容，并不涉及整个prompt
from llama_index.response.notebook_utils import display_source_node
for node in response2.source_nodes:
    display_source_node(node, source_length=20000)
    print("--------")

**Node ID:** 2a4fc196-a4d3-4f88-ab0b-f01da4f74742<br>**Similarity:** 0.8635570923250223<br>**Text:** The 8_1_2015 UNITE ITS fungal sequence database and usearch8.0 was used to remove potentially chimeric sequences (60) (55). The ITSx software package was then used to extract the predicted ITS2 region from the reference sequence of non-chimeric OTUs, filtering out OTUs predicted to lack a true ITS2 region in the process (61). Taxonomy was then assigned to non-chimeric, ITS2 extracted OTUs using Bayesian classification with a confidence cut-off of 0.8 in QIIME according to the 8_1_2015 UNITE ITS fungal sequence database (54) (60). OTUs responsible for less that 0.001% of the total sequence reads were removed. To normalize variation in read depth across samples, data were rarefied to the minimum read depth of 6,653 sequences per sample for bacteria. To ensure that a truly representative community of each sample was used for analysis, sequence sub-sampling at the defined depth was bootstrapped 100 times.<br>

--------


**Node ID:** 6498b965-f6ee-4136-8096-5ea0a99d1ec1<br>**Similarity:** 0.8589275420381168<br>**Text:** Mycologically, NGM3 subjects were consistently depleted of multiple Malassezia taxa (ZINB; Benj amini-Hochberg, q<0.20; FIG. 28 and FIG. 29)-striking, given our population-based observation that this genus is characteristically enriched in the neonatal gut microbiota. Fungal taxonomic enrichments in the NGM3 group were also consistent when compared to either of the lower-risk groups, and included Rhodotorula and Candida (FIG. 28 and FIG. 29). Hence, neonatal interkingdom microbiota dysbiosis is characteristic of PM atopy and asthma development in childhood.NGM3-associated bacterial taxonomic alterations were predicted8 to result in a deficiency in amino acid, lipid and xenobiotic metabolism pathways. Untargeted liquid chromatography mass spectrometry identified fecal metabolites present in a subset (n=28) of the representative subjects from each NGM (those with the highest posterior probability of NGM membership). Substantial correlations existed between the 16S rRNA profile, predicted metagenome and the metabolome of NGMs (Procrustes; FIG.<br>

--------


**Node ID:** bd05a32e-9dcc-425b-a892-fb98279bbf8e<br>**Similarity:** 0.8571140230122907<br>**Text:** In some aspects, the variants or homologs have at least 90%, 95%, 96%, 97%, 98%, 99% or 100% amino acid sequence identity across the whole sequence or a portion of the sequence (e.g. a 50, 100, 150 or 200 continuous amino acid portion) compared to a naturally occurring MUC5B polypeptide. In embodiments, MUC5B is the protein as identified by the UniProt sequence reference Q9HC84 (SEQ ID NO:6), or an isoform, a homolog or functional fragment thereof.In embodiments, the method further includes administering a therapeutically effective amount of a fungus. In embodiments, the fungus is a Malassezia fungus. In embodiments, the microbial composition is effective to decrease a pathogenic fungal activity. A “pathogenic fungal activity” as referred to herein is a metabolic activity derived from a pathogenic fungus. In embodiments, the pathogenic fungus is Candida albicans. In embodiments, the pathogenic fungus forms a pro-inflammatory lipid.In embodiments, the subject is a neonate. In embodiments, the neonate is less than about four weeks old.<br>

--------


**Node ID:** b469fdbd-2b3f-43bd-a9bd-5aa2e4172873<br>**Similarity:** 0.8520443042653374<br>**Text:** within at least 50%, 80%, 90%, 95%, 96%, 97%, 98%, 99% or 100% activity compared to IL-10). In some aspects, the variants or homologs have at least 90%, 95%, 96%, 97%, 98%, 99% or 100% amino acid sequence identity across the whole sequence or a portion of the sequence (e.g. a 50, 100, 150 or 200 continuous amino acid portion) compared to a naturally occurring IL-10 polypeptide. In embodiments, IL-10 is the protein as identified by the NCBI sequence reference GI:10835141 (Accession No. NP_000563.1; SEQ ID NO:2), or an isoform, a homolog or functional fragment thereof.The term “IL-13” as provided herein includes any of the recombinant or naturally-occurring forms of the interleukin 13 (IL-13) cytokine or variants or homologs thereof that maintain IL-13 protein activity (e.g. within at least 50%, 80%, 90%, 95%, 96%, 97%, 98%, 99% or 100% activity compared to IL-13). In some aspects, the variants or homologs have at least 90%, 95%, 96%, 97%, 98%, 99% or 100% amino acid sequence identity across the whole sequence or a portion of the sequence (e.g. a 50, 100, 150 or 200 continuous amino acid portion) compared to a naturally occurring IL-13 polypeptide. In embodiments, IL-13 is the protein as identified by the NCBI sequence reference GI:26787978 (Accession No. NP_002179.2; SEQ ID NO:3), or an isoform, a homolog or functional fragment thereof.The term “IL-17” as provided herein includes any of the recombinant or naturally-occurring forms of the interleukin 17 (IL-17) cytokine or variants or homologs thereof that maintain IL-17 protein activity (e.g. within at least 50%, 80%, 90%, 95%, 96%, 97%, 98%, 99% or 100% activity compared to IL-17).<br>

--------
